In [1]:
import pandas as pd
from requests import get
from bs4 import BeautifulSoup

In [2]:
def get_page(url):
    response = get(url)
    html = response.content
    soup = BeautifulSoup(html, "html.parser")
    return soup

base_ff_url = 'https://www.fleaflicker.com/mlb/leagues/'
league_ids = ['21579', '21581', '21580', '21582', '21583', '21584', '21585', '21586', '21587', '21588', '21589', 
              '21590', '21591', '21592', '21593', '21594', '21595', '21596']

all_teams = []
for l in league_ids:
    url = base_ff_url + l
    soup = get_page(url)
    trs = soup.find_all('tr')
    raw_headers = trs[1].find_all('th')
    player_data = trs[2:]
    headers = []
    for header in raw_headers:
        if header.text:
            headers.append(header.text)
    exp_headers = headers + ['league_id', 'league_name', 'team_id']   
    league_name = soup.find_all('li', {'class': 'active'})[1].text.strip()
    for row in player_data:
        d_dict = dict.fromkeys(exp_headers)
        d_dict['league_id'] = l
        d_dict['league_name'] = league_name
        d_dict['Team'] = row.find('td', {'class': 'left'}).text
        d_dict['Owner'] = row.find('td', {'class': 'right'}).text
        d_dict['team_id'] = row.find('a', href=True).get('href')[-6:]
        try:
            d_dict['Rank'] = row.find_all('td', {'class': 'right text-center'})[-1].text
        except IndexError:
            d_dict['Rank'] = row.find_all('td', {'class': 'bottom right text-center'})[-1].text
        heads = exp_headers[2:14]
        if d_dict['Owner'] == 'Take Over':
            stats = row.find_all('span', {'class': 'nowrap'})
        else:
            stats = row.find_all('span', {'class': 'nowrap'})[1:]
        for h, s in zip(heads, stats):
            d_dict[h] = s.text
        all_teams.append(d_dict)

In [3]:
all_df = pd.DataFrame(all_teams, columns=exp_headers)
all_df.HR = all_df.HR.astype(int)
all_df.R = all_df.R.astype(int)
all_df.RBI = all_df.RBI.astype(int)
all_df.SB = all_df.SB.astype(int)
all_df.OBP = all_df.OBP.astype(float)
all_df.OPS = all_df.OPS.astype(float)
all_df.SO = all_df.SO.astype(int)
all_df.SV = all_df.SV.astype(int)
all_df.HD = all_df.HD.astype(int)
all_df.ERA = all_df.ERA.astype(float)
all_df.WHP = all_df.WHP.astype(float)
all_df.QS = all_df.SV.astype(int)

In [4]:
rank_headers = ['HR', 'R','RBI','SB','OBP','OPS','SO','SV','HD','ERA','WHP','QS']
for r in rank_headers:
    if r in ['ERA', 'WHP']:
        all_df[r+'_Points'] = all_df[r].rank(ascending=False)
    else:
        all_df[r+'_Points'] = all_df[r].rank()

In [5]:
all_df['Total_Points'] = all_df.iloc[:,-12:].sum(axis=1)
all_df['Overall_Rank'] = all_df.Total_Points.rank(ascending=True)

In [7]:
all_df.head()

Team        Owner   HR    R  RBI  SB    OBP    OPS  \
0           Bang Bang Pop Pop   JohnnyFang  116  400  342  35  0.342  0.814   
1           Forearm Tightness  CropRhombus  122  389  360  46  0.340  0.834   
2                Swinger Bell       WMason  120  387  373  32  0.357  0.842   
3  Ten Points for Griffeyndor     coreyjro  106  384  332  25  0.340  0.794   
4           Harper Wallbanger     Prime416  140  391  424  38  0.333  0.820   

    SO  SV  ...  OBP_Points  OPS_Points  SO_Points  SV_Points HD_Points  \
0  724  42  ...       121.0       118.5      200.0      164.5     216.0   
1  653   9  ...       105.0       173.0      133.0       13.0     191.5   
2  692  21  ...       206.0       194.0      169.0       52.0     201.5   
3  703  31  ...       105.0        58.5      184.0      107.0     220.0   
4  701  18  ...        51.0       138.0      181.0       40.0     172.0   

  ERA_Points WHP_Points QS_Points Total_Points  Overall_Rank  
0      136.0      158.5     164.5       1833.0         201.0  
1      104.5      177.5      13.0       1571.0         146.0  
2       73.0      110.0      52.0       1648.5         166.5  
3      164.0      127.0     107.0       1416.0         123.0  
4       80.0       96.0      40.0       1561.0         142.5  

[5 rows x 33 columns]

In [ ]:
#all_df.to_csv('current_rankings.csv')